**18_MAW_iii**

In [1]:
import os ; import pathlib as pl ; import flopy ; import matplotlib.pyplot as plt ; import numpy as np; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl
import pandas as pd ; from modflow_devtools.misc import get_env, timed ; write=get_env("WRITE", True);run=get_env("RUN", True)

In [2]:
sim_ws    = Path("./18_MAW_iii")                    ; sim_ws.mkdir(exist_ok=True) ; sim_name = "ex-gwf-maw-p03"
exe_name  = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe' ; workspace = figs_path = sim_ws ; gwf_name = sim_name ; output_folder = sim_ws   

In [3]:
parameters = {"ex-gwf-maw-p03a": {"simulation": "regional"},
              "ex-gwf-maw-p03b": {"simulation": "multi-aquifer well"},  "ex-gwf-maw-p03c": {"simulation": "high conductivity"}}
def calc_cond(area, l1, l2, k1, k2):
    c1 = area * k1 / l1 ; c2 = area * k2 / l2
    return c1 * c2 / (c1 + c2)
    
nper = 1 ;  nlay_r = 21 ;nrow_r = 1 ;ncol_r = 200      ; maw_highK = 1000000000 ;nouter = 500 ;ninner = 100 ;hclose = 0.000000001 ;rclose = 0.0001
strt = 10 ; nlay = 41   ;nrow = 16  ;ncol = 27 ;delr_r = 50 ;delc_r = 1 ;top = 10 ;aq_bottom = -205 ; k11 = 250 ;k33 = 50 ;recharge = 0.004566 ;
H1 = 0 ;maw_loc = (15,13) ;maw_lay = (1,12) ;maw_radius = 0.1333 ;maw_bot = -65   ; length_units = "feet" ; time_units = "days" ; 
 

delr = [10.0 ,10.0 ,9.002 ,6.0 ,4.0 ,3.0 ,2.0 ,1.33 ,1.25 ,1.00 ,1.00 ,0.75 ,0.50 ,0.333 ,0.50 ,0.75 ,1.00 ,1.00 ,1.25 ,1.333 ,
        2.0 ,3.0 ,4.0 ,6.0 ,9.002 ,10.0 ,10.0]   ; delc = [10, 9.38, 9, 6, 4, 3, 2, 1.33, 1.25, 1, 1, 0.75, 0.5, 0.3735, 0.25, 0.1665]

tdis_ds = ((1.0, 1, 1.0),)
extents = (0.0, np.array(delr).sum(), 0.0, np.array(delc).sum()) ; shape2d = (nrow, ncol) ; shape3d = (nlay, nrow, ncol)
nconn = 2 + 3 * (maw_lay[1] - maw_lay[0] + 1)  ; maw_packagedata = [[0, maw_radius, maw_bot, strt, "SPECIFIED", nconn]]
i, j = maw_loc  ; obs_elev = {}   ; maw_conn = []  ; gwf_obs = []
for k in range(maw_lay[0], maw_lay[1] + 1, 1):       
    gwf_obs.append([f"H0_{k}", "HEAD", (k, i, j)]) ; maw_obs = [["H0", "HEAD", (0,)]]    ; iconn = 0  ; z = -5.0         # obs_dict_B = maw_obs
    
for k in range(maw_lay[0], maw_lay[1] + 1, 1):
    if k == maw_lay[0]:
        area = delc[i] * delr[j]   ; l1 = 2.5            ; l2 = 2.5                 ; cond = calc_cond(area, l1, l2, k33, maw_highK)
        maw_conn.append([0, iconn, k - 1, i, j, top, maw_bot, cond, -999.0])        ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z   
        maw_obs.append([tag, "maw", (0,), (iconn,)])  ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k-1,i,j)]) ; iconn += 1    ; z -= 2.5 # k+1
    area = delc[i] * 5.0           ; l1 = 0.5 * delr[j]  ; l2 = 0.5 * delr[j - 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i, j - 1, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i,j-1)]) ; iconn += 1               # j+1
    area = delr[j] * 5.0           ; l1 = 0.5 * delc[i]  ; l2 = 0.5 * delc[i - 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i - 1, j, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i-1,j)]) ; iconn += 1               # i+1        
    area = delc[i] * 5.0           ; l1 = 0.5 * delr[j]  ; l2 = 0.5 * delr[j + 1]   ; cond = calc_cond(area, l1, l2, maw_highK, k11)
    maw_conn.append([0, iconn, k, i, j + 1, top, maw_bot, cond, -999.0])            ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
    maw_obs.append([tag, "maw", (0,), (iconn,)])      ; gwf_obs.append([tag, "flow-ja-face", (k,i,j), (k,i,j+1)]) ; iconn += 1 ; z -= 5.0    # j+1 (2x)
    if k == maw_lay[1]:
        z += 2.5 ; area = delc[i] * delr[j] ; l1 = 2.5  ; l2 = 2.5                  ; cond = calc_cond(area, l1, l2, maw_highK, k33)
        maw_conn.append([0, iconn, k + 1, i, j, top, maw_bot, cond, -999.0])        ; tag = f"Q{iconn:02d}"       ; obs_elev[tag] = z
        maw_obs.append([tag, "maw", (0,), (iconn,)]) ; gwf_obs.append([tag, "flow-ja-face", (k, i, j), (k + 1, i, j)])     ; iconn += 1
obs_dict2 = maw_obs                                  ; ab=pd.DataFrame(obs_dict2) ; ab.to_csv(output_folder/"06_obs_dict.csv") 
gwf_obs2  = gwf_obs

In [4]:
np.savetxt(output_folder / "01_maw_packagedata.csv", maw_packagedata, fmt ="%s", delimiter=",") 
np.savetxt(output_folder / "02_maw_conn.csv", maw_conn, delimiter=",") 
np.savetxt(output_folder / "03_delr.csv", delr, delimiter=",") ;np.savetxt(output_folder / "04_delc.csv", delc, delimiter=",")    # delr delc
idomain = np.ones(shape3d, dtype=float) ; 
for k in range(maw_lay[0], maw_lay[1] + 1, 1): idomain[k, i, j] = 0
sa = pd.DataFrame(idomain[0,:,:]) ; sa.to_csv (output_folder / "05_idomain.csv") 

In [5]:
#  regional | multi-aquifer well| high conductivity

def build_models(name, simulation="regional"):
    if simulation == "regional": return build_regional(name)
    else:                        return build_local(name, simulation)
def build_regional(name):
    icelltype1 = [1] + [0 for k in range(1, nlay_r)]    ; head_filerecord=f"{sim_name}.hds"   ; botm = np.arange(-5, aq_bottom - 10.0, -10.0)
    sim_ws = os.path.join(workspace, name)
    sim = flopy.mf6.MFSimulation (sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
    flopy.mf6.ModflowTdis (sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
    flopy.mf6.ModflowIms  (sim,print_option="summary",
                           outer_maximum=nouter,outer_dvclose=hclose,inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=f"{rclose} strict")
    gwf = flopy.mf6.ModflowGwf   (sim, modelname=sim_name)
    flopy.mf6.ModflowGwfdis   (gwf,length_units=length_units,nlay=nlay_r,nrow=nrow_r,ncol=ncol_r,delr=delr_r,delc=delc_r,top=top,botm=botm)
    flopy.mf6.ModflowGwfnpf   (gwf,icelltype=icelltype1,k=k11,k33=k33)
    flopy.mf6.ModflowGwfic    (gwf,strt=strt)
    flopy.mf6.ModflowGwfchd   (gwf,stress_period_data=[[0,0,ncol_r-1,0.0]])
    flopy.mf6.ModflowGwfrcha  (gwf,recharge=recharge)
    flopy.mf6.ModflowGwfoc    (gwf,head_filerecord=head_filerecord,saverecord=[("HEAD","LAST")],printrecord=[("BUDGET","LAST")])
    return sim
    
def build_local(name, simulation):
    pth = next(iter(parameters.keys())) ; fpth = os.path.join(workspace, pth, f"{sim_name}.hds")
    try:     h = flopy.utils.HeadFile(fpth).get_data()    
    except:  h = np.ones((nlay_r, nrow_r, ncol_r), dtype=float) * strt
    f1 = 0.5 * (delr_r + delr[0]) / delc_r ; f2 = 0.5 * (delr_r + delr[-1]) / delc_r
    regional_range = [k for k in range(nlay_r)]      ; local_range = [[0]] + [[k, k + 1] for k in range(1, nlay, 2)]    ; chd_spd = []
    for kr, kl in zip(regional_range, local_range):
        h1 = h[kr ,0 ,0];h2=h[kr ,0 ,1]              ; hi1=h1+f1*(h2-h1);h1=h[kr ,0 ,2]            ; h2=h[kr ,0 ,3] ; hi2=h1+f2*(h2-h1)
        for k in kl:
            for il in range(nrow): chd_spd.append([k, il, 0, hi1])                             ; chd_spd.append([k, il, ncol - 1, hi2])

    botm = np.arange(-5, aq_bottom - 5.0, -5.0)  ; icelltype2 = [1] + [0 for k in range(1, nlay)]     ; i, j = maw_loc
    if simulation == "multi-aquifer well":
        k11_sim = k11          ; k33_sim = k33   ; idomain = np.ones(shape3d, dtype=float)
        for k in range(maw_lay[0], maw_lay[1] + 1, 1): idomain[k, i, j] = 0
    else:
        k11_sim = np.ones(shape3d, dtype=float) * k11    ; k33_sim = np.ones(shape3d, dtype=float) * k33 ; idomain = 1
        for k in range(maw_lay[0], maw_lay[1] + 1, 1):   
            k11_sim[k, i, j] = maw_highK                 ; k33_sim[k, i, j] = maw_highK

    sim_ws = os.path.join(workspace, name)
    sim = flopy.mf6.MFSimulation (sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
    flopy.mf6.ModflowTdis (sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
    flopy.mf6.ModflowIms  (sim,print_option="summary",outer_maximum=nouter,outer_dvclose=hclose,
                           inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=f"{rclose} strict")
    gwf = flopy.mf6.ModflowGwf   (sim, modelname=sim_name, save_flows=True)
    flopy.mf6.ModflowGwfdis   (gwf,length_units=length_units,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=top,botm=botm,idomain=idomain)
    flopy.mf6.ModflowGwfnpf   (gwf,icelltype=icelltype2, k=k11_sim, k33=k33_sim, save_specific_discharge=True)
    flopy.mf6.ModflowGwfic    (gwf, strt=strt)
    flopy.mf6.ModflowGwfchd   (gwf, stress_period_data=chd_spd)
    flopy.mf6.ModflowGwfrcha  (gwf, recharge=recharge)
    if simulation == "multi-aquifer well":
        maw = flopy.mf6.ModflowGwfmaw(gwf ,no_well_storage=True ,nmawwells=1 ,packagedata=maw_packagedata ,connectiondata=maw_conn)
        
        obs_file = f"{sim_name}.maw.obs" ; csv_file = obs_file +".csv" ; obs_dict = { csv_file: maw_obs}
        maw.obs.initialize(filename=obs_file ,digits = 10 ,print_input=True ,continuous=obs_dict)
    else:
        obs_file=f"{sim_name}.gwf.obs"   ; csv_file = obs_file+".csv" ; obsdict={csv_file:gwf_obs}
        
        flopy.mf6.ModflowUtlobs(gwf ,filename=obs_file ,print_input=False ,continuous=obsdict)
        head_filerecord=f"{sim_name}.hds"  ; budget_filerecord=f"{sim_name}.cbc"
        flopy.mf6.ModflowGwfoc(gwf ,head_filerecord=head_filerecord ,budget_filerecord=budget_filerecord ,
                               saverecord=[("HEAD" ,"LAST") ,("BUDGET" ,"LAST")] ,printrecord=[("BUDGET" ,"LAST")])
    return sim
def write_models(sim, silent=True):
    sim.write_simulation(silent=silent)
@timed
def run_models(sim, silent=True):
    success, buff = sim.run_simulation(silent=silent)  ; assert success, buff
def scenario(idx=0, silent=True):
    key = list(parameters.keys())[idx]
    params = parameters[key].copy()
    sim = build_models(key, **params)
    if write:
        write_models(sim, silent=silent)
    if run:
        run_models(sim, silent=silent)
scenario(0); scenario(1); scenario(2)

run_models took 203.56 ms
run_models took 264.17 ms
run_models took 360.50 ms


In [6]:
maw0 = pd.read_csv(sim_ws / 'ex-gwf-maw-p03b\\ex-gwf-maw-p03.maw.obs.csv')    
maw1 = pd.read_csv(sim_ws / 'ex-gwf-maw-p03c\\ex-gwf-maw-p03.gwf.obs.csv')    

In [7]:
maw0; maw1

,time,H0_1,H0_2,H0_3,H0_4,H0_5,H0_6,H0_7,H0_8,H0_9,...,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37
0,1.0,4.925856,4.925856,4.925856,4.925855,4.925855,4.925855,4.925854,4.925854,4.925853,...,-0.161503,-0.470759,-0.251215,-0.223528,-0.612259,-0.313242,-0.31048,-0.810921,-0.400222,-0.002185


In [8]:
sim_ws2 = os.path.join(workspace, 'ex-gwf-maw-p03b') ; sim_ws3 = os.path.join(workspace, 'ex-gwf-maw-p03c') 
sim2 = flopy.mf6.MFSimulation.load(sim_ws = sim_ws2 ,exe_name = exe_name ,verbosity_level = 0) 
sim3 = flopy.mf6.MFSimulation.load(sim_ws = sim_ws3 ,exe_name = exe_name ,verbosity_level = 0) 